# koGPT-2 모델 학습

### 데이터 불러오기

import

In [ ]:
import pandas as pd
import tqdm

드라이버 연결

In [ ]:
from google.colab import drive
import os, shutil

drive.mount('/content/drive/')
os.chdir('drive/My Drive/dataset')
!pwd

#### csv 가져오기
데이터 : Q,A 나눠져있는 데이터


In [ ]:
import pandas as pd
import tqdm

train 데이터 불러오기

In [ ]:
train_data = pd.read_csv("all_chat.csv")

GPT에서 사용하지 않는 [SEP] 토큰 제거

In [ ]:
train_data['Q'] = train_data['Q'].apply(lambda x: x.replace('[SEP]',''))
train_data['Q'].iloc[-1]

'요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나.자식들과 아내 때문에 마음이 편치 않으시군요.전에는 그러지 않았는데 요즘은 점점 돈이나 게임기나 옷이나 나한테 요구하기만 해.이 상황을 극복하려면 어떻게 해야 할까요?가족들과 한번 진중한 대화를 나눠봐야겠어.진중한 대화를 나눠보면 어떤 점이 달라질 수 있을까요?최소한 내가 스트레스 받고 있다는 건 알릴 수 있을 것 같네.'

In [ ]:
train_data.head()

,Q,A,연령,성별,상황키워드,감정_대분류,감정_소분류
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,0,0,0,0,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,1,0,1,1,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,2,1,2,2,2
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,1,0,3,0,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,1,1,3,0,3


비슷한 성격의 wellness 데이터 불러오기

In [ ]:
well = pd.read_csv('wellness_dataset.csv')
print(well.구분.nunique())
well.head(2)

355


,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806065, -0.29486918, 0.43789992, -0.64013..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561579, -0.14506263, 0.29490355, -0.67394..."


In [ ]:
well=well[['유저','챗봇']]
well.rename(columns = {'유저' : 'Q','챗봇' : 'A'}, inplace = True)
well.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Q,A
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?


train과 wellness를 합쳐서 전체적인 학습 데이터 생성

In [ ]:
train_data = pd.concat([train_data,well],ignore_index=True)

In [ ]:
train_data = train_data.sample(frac=1).reset_index(drop=True)

In [ ]:
# train_data.drop(['Unnamed: 0'], axis=1, inplace= True)

In [ ]:
train_data.tail()

,Q,A,연령,성별,상황키워드,감정_대분류,감정_소분류
129476,아들이 면접에서 떨어졌는데 너무 슬퍼.아들의 면접 결과가 좋지 않아 속상하셨군요.응...,붙을 거라 기대를 했는데 떨어져서 낙담하셨겠어요.,3.0,1.0,11.0,3.0,15.0
129477,나 이제 따돌림에서 벗어났어! 너무 기뻐!따돌림에서 벗어나셔서 기쁘군요.전학 온 친...,전학 온 친구분께 고마움을 느끼고 계시는군요. 어떻게 하면 친구분께 보답할 수 있을까요?,2.0,1.0,6.0,0.0,29.0
129478,아직은 내 나이가 이십 대의 중반이라 다행이라고 생각해.,왜 다행이라고 생각하세요?,0.0,1.0,0.0,0.0,3.0
129479,다리가 아파 병원에 갔더니 허리랑 위도 위험하다고 해서 속상해.저런 건강이 많이 안...,아픈 곳이 계속 생겨 병원에 다니시는 것이 고민이 되시는군요.,3.0,1.0,5.0,3.0,31.0
129480,내 가게가 티브이에 나와서 손님들이 북새통을 이루고 있어서 너무 기뻐.축하해요! 너...,미디어의 힘이 대단하다고 생각하시네요. 이 상황에서 할 수 있는 게 어떤 것이 있을까요?,1.0,0.0,3.0,0.0,0.0


### 토크나이저
 KoGTP-2의 모델과 토크나이저를 불러오기

####  KoGTP-2 모델 로드
KoGTP-2 모델과 토크나이저를 trainsformers를 이용하여 로드

스페셜 토큰들
- `<usr>` 발화 토큰을 question 시작 토큰으로 사용
- `<sys>` 대답 토큰을 answer 시작 토큰으로 사용
- `<pad>` 패딩 토큰을 패딩용으로 사용

In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

####  스페셜 토큰을 이용한 정수화
위에 지정한 스페셜 토큰을 이용하여 question, answer 문자에 대한 전처리와 정수화를 진행한 tesnsor flow Dataset을 생성

In [ ]:
batch_size = 8

get_chat_data 함수는:
1. train_data의 Q와 A를 읽어 각각 한 문장씩 다음 작업을 반복합니다.
2. `'<usr>'` + question + `'<sys>'` + answer로 부착해서 정수화 sent를 생성하고
3. sent 앞뒤로 bos_token_id와 eos_token_id를 추가합니다.
4. 매 반복마다 sent를 출력하는 generator를 생성합니다.

In [ ]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    question.replace('\t',' ')
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer) 
    yield bos_token + sent + eos_token

In [ ]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [ ]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

### 모델 학습

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [ ]:
len(train_data)

129481

In [ ]:
steps = len(train_data) // batch_size + 1
print(steps)

16186


3 에포크 학습:

In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/16186 [00:00<?, ?it/s]

[Epoch:    1] cost = 1.38326752


  0%|          | 0/16186 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.00982189


...

[Epoch:    3] cost = 0.711592019


  0%|          | 0/16186 [00:00<?, ?it/s]

[Epoch:    4] cost = 0.511387289


  0%|          | 0/16186 [00:00<?, ?it/s]

[Epoch:    5] cost = 0.391855329


### 모델 저장

In [ ]:
model.save_pretrained('GPTmodel_well_3E_b8.h5')

저장된 모델 로딩

In [ ]:
m = TFGPT2LMHeadModel.from_pretrained('GPTmodel_well_3E_b8.h5')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./save/sentGenModel.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
